## 1, Import Library

In [3]:
from machine_lib_output import *
from capm_alpha import *
from capm_alpha_opt import *
from alpha_template import *

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key="fb7dad15-9df1-4ffd-bb51-058790ef27fe", # 请替换成您的ModelScope SDK Token
    base_url="https://api-inference.modelscope.cn/v1/"
)


response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-V3", # ModleScope Model-Id
    messages=[
        {
            'role': 'system',
            'content': 'You are a helpful assistant.'
        },
        {
            'role': 'user',
            'content': '用python写一下快排'
        }
    ],
    stream=True
)

for chunk in response:
    print(chunk.choices[0].delta.content, end='', flush=True)

## 2, 登录
<div style="margin-left: 20px;">
1, 在machine_lib文件的login方法中填写用户名和密码后保存，然后来到本文件Restart Kernal后重新import machine_lib后才在本文件生效
</div>

<div style="margin-left: 20px;">
2, 打印INVALID_CREDENTAIL即登录失败，打印自己的user_id信息才是登录成功。
</div>

In [ ]:
s = login()

### 二阶提升

In [14]:
fo_tracker = get_alphas_from_csv("output/simulated_alphas_ASI.csv.2025-07-17", 1.2, 0.7)
print(len(fo_tracker))

filtered_list = [
    item for item in fo_tracker
    if all(x not in item[1] for x in ['sentiment', 'trade']) 
    and sum(item[1].lower().count(keyword) for keyword in ['group_rank', 'group_scale', 'group_count', 'group_zscore', 'group_std_dev', 'group_sum', 'group_neutralize']) < 2
]
print(len(filtered_list))
print(filtered_list[:3])

fo_layer = digest(filtered_list)
# 剪枝后数量
print(len(fo_layer))
print(fo_layer[:3])

so_alpha_list = []
group_ops = ['group_rank', 'group_scale', 'group_count', 'group_zscore', 'group_std_dev', 'group_sum', 'group_neutralize']

for expr, decay in fo_layer:
#for expr, decay in fo_exp_layer:
    #for alpha in get_group_second_order_factory([expr], group_ops, "EUR"):
    for alpha in get_group_second_order_factory_for_multi_line([expr], group_ops, "GLB"):
        so_alpha_list.append((alpha,decay))

random.shuffle(so_alpha_list)
print(len(so_alpha_list))
print(so_alpha_list[:3])

8
0
[]
0
[]
0
[]


In [13]:
generate_pending_simulation_data(so_alpha_list,"CROWDING", "GLB", "TOPDIV3000", "OFF", "append", 'alpha_list_pending_simulated_1.csv')

All simulation data written to output\alpha_list_pending_simulated_1.csv in append mode


### 三阶提升

In [ ]:
## get promising alphas from second order to improve in the third order
#so_tracker = get_alphas("04-22", "04-23", 1.3, 0.8, "ASI", 5000, "track")
so_tracker = get_alphas_from_csv("output/simulated_alphas_EUR.csv.2025-07-07", 1.3, 0.8)

filtered_list = [
    item for item in so_tracker
    if all(x not in item[1] for x in ['sentiment', 'trade'])
]
print(len(filtered_list))
print(filtered_list[:3])
so_layer = digest(filtered_list)
th_alpha_list = []


for expr, decay in so_layer:
    for alpha in trade_when_factory("trade_when",expr,"EUR"):
        th_alpha_list.append((alpha,decay))

random.shuffle(th_alpha_list)        
print("三阶表达式数量:%s"%len(th_alpha_list))

24
[['V5Kox8J', '-ts_std_dev(winsorize(ts_backfill(cashflow_op /assets, 120), std=4)/group_mean(winsorize(ts_backfill(cashflow_op /assets, 120), std=4), 1, subindustry), 240)', -1.37, 0.0246, -1.5, -0.012257, '2025-07-05T22:30:20-04:00', 6], ['RK00dP1', '-ts_zscore(winsorize(ts_backfill(fnd6_np / star_val_earnings_projection_fy3, 120), std=4), 66)', -1.45, 0.1301, -0.85, -0.000679, '2025-07-05T22:45:03-04:00', 6], ['QgmE6Wp', 'ts_mean(winsorize(ts_backfill(cashflow_op /assets, 120), std=4), 22)', 1.82, 0.0251, 1.49, 0.006711, '2025-07-05T23:51:55-04:00', 6]]
三阶表达式数量:1632


In [5]:
generate_pending_simulation_data(th_alpha_list,"SUBINDUSTRY",  "EUR", "TOP2500", "OFF", "append", 'alpha_list_pending_simulated_1.csv')

All simulation data written to output\alpha_list_pending_simulated_1.csv in append mode


### mutil-line二阶提升

In [ ]:
fo_tracker = get_alphas_from_csv("output/simulated_alphas_USA.csv.2025-06-10", 1.2, 0.7)
print(len(fo_tracker))

filtered_list = [
    item for item in fo_tracker
    if 'sentiment' in item[1] and 'trade' not in item[1]
]
print(len(filtered_list))
print(filtered_list[:3])
fo_layer = digest(filtered_list)
# 剪枝后数量
print(len(fo_layer))
print(fo_layer[:3])

so_alpha_list = []
group_ops = ["group_neutralize", "group_rank", "group_zscore"]

for expr, decay in fo_layer:
#for expr, decay in fo_exp_layer:
    for alpha in get_group_second_order_factory_for_multi_line([expr], group_ops, "USA"):
        so_alpha_list.append((alpha,decay))

random.shuffle(so_alpha_list)
print(len(so_alpha_list))
print(so_alpha_list[:3])

In [ ]:
generate_pending_simulation_data(so_alpha_list,"SUBINDUSTRY", "USA", "TOP3000", "OFF", "append", 'alpha_list_pending_simulated_1.csv')

### mutil-line三阶提升

In [ ]:
so_tracker = get_alphas_from_csv("output/simulated_alphas_USA.csv.2025-06-10", 1.3, 0.8)

filtered_list = [
    item for item in fo_tracker
    if 'sentiment' in item[1] and 'trade' not in item[1]
]
print(len(filtered_list))
print(filtered_list[:3])
so_layer = digest(filtered_list)
th_alpha_list = []
for expr, decay in so_layer:
    for alpha in trade_when_factory_for_multi_line("trade_when",expr,"USA"):
        th_alpha_list.append((alpha,decay))

random.shuffle(th_alpha_list)        
print("三阶表达式数量:%s"%len(th_alpha_list))

In [ ]:
generate_pending_simulation_data(th_alpha_list,"SUBINDUSTRY", "USA", "TOP3000", "OFF", "append", 'alpha_list_pending_simulated_1.csv')

## 3, 获取数据字段
<div style="margin-left: 20px;">
在官网Data页面中显示的为自己目前有权限的数据集，在数据集Description面板下可以看到dataset_id
</div>

In [ ]:


import re

df = get_datafields(s, dataset_id = 'fundamental17', region='USA', universe='TOP3000', delay=1)
"""
target_field = df[df['description'].str.contains('EPS|Profit|Revenue|Income|Earning|Margin')]["id"].values.tolist()
print(target_field)
print(len(target_field))

anl15_df = get_datafields(s, dataset_id = 'analyst15', region='USA', universe='TOP3000', delay=1)
market_field = anl15_df[anl15_df['id'].str.contains('mean|pe|total|gro')]["id"].values.tolist()
market_field = [id_ for id_ in market_field if re.search('anl15_gr_|anl15_ind_|anl15_s_', id_)]
print(market_field[:10])
print(len(market_field))

sector_field = ['sta1_allc5', 'sta1_allc10', 'sta1_allc20', 'sta1_top3000c10', 'sta1_top500c10']
"""

In [ ]:
target_fields_v = [
    "nws18_acb", "nws18_bam", "nws18_bee", "nws18_ber", "nws18_qcm", "nws18_qep", "nws18_ssc", "nws18_sse"
]
target_field_m = [
    "rp_css_earnings", "rp_css_revenue", "rp_ess_earnings", "rp_ess_revenue", "rp_nip_earnings"
]
target_fields = get_vec_fields(target_fields_v) + target_field_m
print(len(target_fields))

news36_df = get_datafields(s, dataset_id = 'news36', region='USA', universe='TOP3000', delay=1)
market_field_v = news36_df[news36_df['id'].str.contains('positive|negative|event')]["id"].values.tolist()
market_fields = get_vec_fields(market_field_v)
print(len(market_fields))
sector_fields = [
    "sta1_allc10", "sta1_allc20", "sta1_top3000c10", "sta1_top500c10"
]
news_days = [252]

In [ ]:
exp_list = read_txt_to_list('data_set/GLBD1.txt')


        

recommended = doubao_exp + recommended_ids + recommended_exp
list = get_ids_from_csv_directory('data/USA', "*_matrix.csv")
exp_list_filter = filter_expressions_by_list_b(recommended, list)

recommended_vec = recommended_vec_exp + recommended_vec_id
list_vec = get_ids_from_csv_directory('data/USA', "*_vector.csv")
exp_list_filter_vec = filter_expressions_by_list_b(recommended_vec, list_vec)
pc_fields = process_list_datafields(exp_list_filter + exp_list_filter_vec)
pc_fields += generate_group_mean_datafield(exp_list_filter + exp_list_filter_vec)
pc_fields += generate_group_mean_datafield_2(exp_list_filter + exp_list_filter_vec)
print(f"exp_list len {len(recommended + recommended_vec)}")
print(f"exp_list_filter len {len(exp_list_filter + exp_list_filter_vec)}")
print(f"pc_fields len {len(pc_fields)}")
print(pc_fields[:10])
#pc_fields += other_exp
print(pc_fields[-10:])



Successfully read 2312 lines from data_set/GLBD1.txt
从 data/USA/model11_matrix.csv 提取 1 个 id: ['mdl11_digital_revenue_signal']
从 data/USA/model26_matrix.csv 提取 545 个 id: ['mdl26_5yr_hstrcl_grwth_rt', 'mdl26_60dy_srprs_lst_q_rnngs', 'mdl26_60dy_srprs_lst_y_rnngs', 'mdl26_7dy_bld_stmt_flg_fq1_rnngs', 'mdl26_7dy_bld_stmt_flg_fq2_rnngs', 'mdl26_7dy_bld_stmt_flg_fq3_rnngs', 'mdl26_7dy_bld_stmt_flg_fq4_rnngs', 'mdl26_7dy_bld_stmt_flg_fy2_rnngs', 'mdl26_7dy_mn_chng_flg_fq1_rnngs', 'mdl26_7dy_mn_chng_flg_fq2_rnngs', 'mdl26_7dy_mn_chng_flg_fq3_rnngs', 'mdl26_7dy_mn_chng_flg_fq4_rnngs', 'mdl26_7dy_mn_chng_flg_fy1_rnngs', 'mdl26_7dy_mn_chng_flg_fy2_rnngs', 'mdl26_actual_last_q_earnings', 'mdl26_actual_last_q_ebitda', 'mdl26_actual_last_q_revenue', 'mdl26_actual_last_y_earnings', 'mdl26_actual_last_y_ebitda', 'mdl26_actual_last_y_revenue', 'mdl26_arm_score_change_1', 'mdl26_arm_score_change_3', 'mdl26_arm_score_change_60', 'mdl26_arm_score_change_7', 'mdl26_arm_score_change_90', 'mdl26_avrevsn_fq1

## 4，数据字段预处理

<div style="margin-left: 20px;">
1, matrix, vector 数据类型
</div>

<div style="margin-left: 20px;">
2, ts_backfill 回填缺失值，提高数据Coverage 
</div>

<div style="margin-left: 20px;">
2, winsorize 去极值
</div>

In [ ]:
pc_fields = process_datafields(df)
len(pc_fields)

In [12]:
pc_fields = parse_alpha_blocks("data_set\\decoded_expressions_shrt7.txt", 3)
len(pc_fields)
print(pc_fields[:10])
pc_fields1 = parse_alpha_blocks("data_set\\decoded_expressions_shrt7_exp.txt", 3)
len(pc_fields1)
print(pc_fields1[:10])

pc_fields2 = parse_alpha_blocks("data_set\\decoded_expressions_news20.txt", 3)
len(pc_fields2)
print(pc_fields2[:10])
pc_fields3 = parse_alpha_blocks("data_set\\decoded_expressions_anl15.txt", 3)
len(pc_fields3)
print(pc_fields3[:10])
pc_fields4 = parse_alpha_blocks("data_set\\decoded_expressions_fnd17.txt", 3)
len(pc_fields4)
print(pc_fields4[:10])
pc_fields5 = parse_alpha_blocks("data_set\\decoded_expressions_fnd17(2).txt", 3)
len(pc_fields5)
print(pc_fields5[:10])
pc_fields.extend(pc_fields1)
pc_fields.extend(pc_fields2)
pc_fields.extend(pc_fields3)
pc_fields.extend(pc_fields4)
pc_fields.extend(pc_fields5)
print(len(pc_fields))
print(pc_fields[:10])

Found 14560 valid alpha blocks in data_set\decoded_expressions_shrt7.txt
['financial_data = ts_backfill(vec_sum(shrt7_combo_deltarate), 60);\ngp = group_cartesian_product(country, industry);\nts_zscore(group_std_dev(financial_data, gp), 20)', 'financial_data = ts_backfill(vec_sum(shrt7_combo_deltarate), 60);\ngp = group_cartesian_product(country, industry);\nts_zscore(group_std_dev(financial_data, gp), 60)', 'financial_data = ts_backfill(vec_sum(shrt7_combo_deltarate), 60);\ngp = group_cartesian_product(country, industry);\nts_zscore(group_std_dev(financial_data, gp), 90)', 'financial_data = ts_backfill(vec_sum(shrt7_combo_deltarate), 60);\ngp = group_cartesian_product(country, industry);\nts_zscore(group_std_dev(financial_data, gp), 200)', 'financial_data = ts_backfill(vec_sum(shrt7_combo_deltarate), 60);\ngp = group_cartesian_product(country, industry);\nts_zscore(group_sum(financial_data, gp), 20)', 'financial_data = ts_backfill(vec_sum(shrt7_combo_deltarate), 60);\ngp = group_carte

## 5, Alpha factory 
<div style="margin-left: 20px;">
在factory方法中将数据字段与操作符组装成alpha表达式
</div>

In [ ]:
first_order = first_order_factory(pc_fields, ts_ops)
print(first_order[:10])
print(len(first_order))

['winsorize(ts_backfill((debt/equity), 120), std=4)', 'ts_rank(winsorize(ts_backfill((debt/equity), 120), std=4), 5)', 'ts_rank(winsorize(ts_backfill((debt/equity), 120), std=4), 22)', 'ts_rank(winsorize(ts_backfill((debt/equity), 120), std=4), 66)', 'ts_rank(winsorize(ts_backfill((debt/equity), 120), std=4), 120)', 'ts_rank(winsorize(ts_backfill((debt/equity), 120), std=4), 240)', 'ts_zscore(winsorize(ts_backfill((debt/equity), 120), std=4), 5)', 'ts_zscore(winsorize(ts_backfill((debt/equity), 120), std=4), 22)', 'ts_zscore(winsorize(ts_backfill((debt/equity), 120), std=4), 66)', 'ts_zscore(winsorize(ts_backfill((debt/equity), 120), std=4), 120)']
71736


In [ ]:
first_order = generate_alpha_list(target_fields, market_fields, sector_fields, cap, news_days, ['residual_alpha','beta_alpha'])
print(first_order[:10])
print(len(first_order))

In [ ]:
first_order = generate_alpha_list(target_list, market_list, sector_list, cap, days, ['residual_alpha'])
print(first_order[:10])
print(len(first_order))

In [ ]:
first_order = generate_alpha_list(
    base_target_list, 
    base_market_list, 
    cap_grouping_fields, 
    cap, 
    days, 
    ['base_alpha', 'base_rank_alpha'])
print(first_order[:10])
print(len(first_order))

In [ ]:

recommended_ids = [
    
]



recommended = recommended_exp + recommended_ids
#recommended = recommended_vec_exp + recommended_vec_id
list = get_ids_from_csv_directory('data/USA', "*_vector.csv")
exp_list_filter = filter_expressions_by_list_b(recommended, list)
#pc_fields = process_list_datafields(exp_list_filter)
print(f"exp_list len {len(recommended)}")
print(f"exp_list_filter len {len(exp_list_filter)}")

#df = pd.read_csv("data/USA/analyst15_matrix.csv")
#data = df[["id", "description"]].to_dict("records")
#expressions = generate_field_expressions(data, suffix1="_mean", suffix2="_st_dev", operator="/")
#print(len(expressions))
#print(expressions)
#filtered_fields = filter_fields_by_keywords(data, keywords=["_m_mean", "_m_gro", "_m_gro", "_m_st_dev"],minuskey=[])
filtered_fields = exp_list_filter
print(len(filtered_fields))
print(filtered_fields)
field = filtered_fields
"""
group1 = get_group_by_region('EUR')
print(len(group1))
group2 = get_group_by_region('')
print(len(group2))
days = ['63', '252', ]
"""
cap_group = "bucket(rank(cap), range='0,0.1,0.1')"
asset_group = "bucket(rank(assets),range='0,0.1,0.1')"
sector_cap_group = "bucket(group_rank(cap, sector),range='0,0.1,0.1')"
sector_asset_group = "bucket(group_rank(assets, sector),range='0,0.1,0.1')"

vol_group = "bucket(rank(ts_std_dev(returns,20)),range='0,0.1,0.1')"

liquidity_group = "bucket(rank(close*volume),range='0,0.1,0.1')"
groups = get_ids_from_csv_directory('data/USA', "other455_illiquid_group.csv")
print(len(groups))
print(groups[:10])

#groups = [cap_group, asset_group, sector_cap_group, sector_asset_group, vol_group, liquidity_group]
#groups = [cap_group, sector_cap_group, vol_group, liquidity_group]

first_order = generate_sentiment_alpha_expressions(field, groups)
print(len(first_order))
print(first_order[:10])


In [ ]:
eur_group_13 = ['pv13_5_sector', 'pv13_2_sector', 'pv13_v3_3l_scibr', 'pv13_v3_2l_scibr', 'pv13_2l_scibr',
                    'pv13_52_sector', 'pv13_v3_6l_scibr', 'pv13_v3_4l_scibr', 'pv13_v3_1l_scibr']
    
eur_group_1 = ['sta1_allc10', 'sta1_allc2', 'sta1_top1200c2', 'sta1_allc20', 'sta1_top1200c10']
    
eur_group_2 = ['sta2_top1200_fact3_c50','sta2_top1200_fact3_c20','sta2_top1200_fact4_c50']

group_field = eur_group_13 + eur_group_1 + eur_group_2

list = get_ids_from_csv_directory('data/EUR', "*_group.csv")
exp_list_filter = filter_expressions_by_list_b(group_field, list)
#pc_fields = process_list_datafields(exp_list_filter)
print(f"exp_list len {len(group_field)}")
print(f"exp_list_filter len {len(exp_list_filter)}")


## 6, 回测前载入

<div style="margin-left: 20px;">
1, alpha表达式与初始decay配对
</div>

<div style="margin-left: 20px;">
2, random shuffle 
</div>

<div style="margin-left: 20px;">
2, Load task pool数据结构
</div>

In [13]:
# 赋予alpha表达式一个初始decay
init_decay = 6
fo_alpha_list = []

for alpha in pc_fields:
    fo_alpha_list.append((alpha, init_decay))

# 随机采样快速评估一个数据集的潜力
random.shuffle(fo_alpha_list)

print("数量: %s"%len(fo_alpha_list))
print(fo_alpha_list[:5])

数量: 168784
[('financial_data = winsorize(ts_backfill(fnd17_alldelay1_apr2rev, 60), std=4);\ngp = group_cartesian_product(country, industry);\nts_rank(group_scale(financial_data, gp), 90)', 6), ('financial_data = winsorize(ts_backfill(fnd17_alldelay1_vol1dprc, 60), std=4);\ngp = group_cartesian_product(country, industry);\nts_delta(group_sum(financial_data, gp), 200)', 6), ('financial_data = winsorize(ts_backfill(fnd17_divtrendgr, 60), std=4);\ngp = group_cartesian_product(country, industry);\nts_returns(group_neutralize(financial_data, gp), 90)', 6), ('financial_data = winsorize(ts_backfill(anl15_ind_18_m_mktcap, 60), std=4);\ngp = group_cartesian_product(country, industry);\nts_av_diff(group_std_dev(financial_data, gp), 60)', 6), ('financial_data = winsorize(ts_backfill(fnd17_alldelay1_divnqxdt, 60), std=4);\ngp = group_cartesian_product(country, industry);\nts_count_nans(group_count(financial_data, gp), 200)', 6)]


In [ ]:
# 赋予alpha表达式一个初始decay
init_decay = 6
fo_alpha_list_usa = []

for alpha in first_order:
    fo_alpha_list_usa.append((alpha, init_decay))

# 随机采样快速评估一个数据集的潜力
random.shuffle(fo_alpha_list_usa)

print("数量: %s"%len(fo_alpha_list_usa))
print(fo_alpha_list_usa[:5])

In [14]:
generate_pending_simulation_data(fo_alpha_list[:50000],"CROWDING", "GLB", "TOPDIV3000", "OFF", "overwrite", "alpha_list_pending_simulated_2_glb.csv")
generate_pending_simulation_data(fo_alpha_list[50000:],"CROWDING", "GLB", "TOPDIV3000", "OFF", "overwrite", "alpha_list_pending_simulated_2_glb_remain.csv")

"""
generate_pending_simulation_data(fo_alpha_list[30000:60000],"SUBINDUSTRY", "USA", "TOP3000", "OFF", "overwrite", "alpha_list_pending_simulated_8.csv")
generate_pending_simulation_data(fo_alpha_list[60000:90000],"SUBINDUSTRY", "USA", "TOP3000", "OFF", "overwrite", "alpha_list_pending_simulated_10.csv")
generate_pending_simulation_data(fo_alpha_list[90000:120000],"SUBINDUSTRY", "USA", "TOP3000", "OFF", "overwrite", "alpha_list_pending_simulated_12.csv")
generate_pending_simulation_data(fo_alpha_list[120000:150000],"SUBINDUSTRY", "USA", "TOP3000", "OFF", "overwrite", "alpha_list_pending_simulated_14.csv")
generate_pending_simulation_data(fo_alpha_list[150000:180000],"SUBINDUSTRY", "USA", "TOP3000", "OFF", "overwrite", "alpha_list_pending_simulated_15.csv")
generate_pending_simulation_data(fo_alpha_list[180000:210000],"SUBINDUSTRY", "USA", "TOP3000", "OFF", "overwrite", "alpha_list_pending_simulated_16.csv")
generate_pending_simulation_data(fo_alpha_list[210000:],"SUBINDUSTRY", "USA", "TOP3000", "OFF", "overwrite", "alpha_list_pending_simulated_17.csv")
generate_pending_simulation_data(fo_alpha_list[100000 : 150000],"SUBINDUSTRY", "ASI", "MINVOL1M", "overwrite", "alpha_list_pending_simulated_4.csv")
generate_pending_simulation_data(fo_alpha_list[150000 : 200000],"SUBINDUSTRY", "ASI", "MINVOL1M", "overwrite", "alpha_list_pending_simulated_5.csv")
generate_pending_simulation_data(fo_alpha_list[200000 : 250000],"SUBINDUSTRY", "ASI", "MINVOL1M", "overwrite", "alpha_list_pending_simulated_6.csv")
generate_pending_simulation_data(fo_alpha_list[250000 : 300000],"SUBINDUSTRY", "ASI", "MINVOL1M", "overwrite", "alpha_list_pending_simulated_10.csv")
generate_pending_simulation_data(fo_alpha_list[300000 :],"SUBINDUSTRY", "ASI", "MINVOL1M", "overwrite", "alpha_list_pending_simulated_11.csv")

generate_pending_simulation_data(fo_alpha_list[:6],"SUBINDUSTRY", "ASI", "MINVOL1M", "overwrite")
generate_pending_simulation_data(fo_alpha_list_usa[:6],"SUBINDUSTRY", "USA", "TOP3000", "append")
generate_pending_simulation_data(fo_alpha_list[7:12],"SUBINDUSTRY", "ASI", "MINVOL1M", "append")
generate_pending_simulation_data(fo_alpha_list_usa[7:12],"SUBINDUSTRY", "USA", "TOP3000", "append")
generate_pending_simulation_data(fo_alpha_list[13:18],"SUBINDUSTRY", "ASI", "MINVOL1M", "append")
generate_pending_simulation_data(fo_alpha_list_usa[13:18],"SUBINDUSTRY", "USA", "TOP3000", "append")
generate_pending_simulation_data(fo_alpha_list[19:24],"SUBINDUSTRY", "ASI", "MINVOL1M", "append")
generate_pending_simulation_data(fo_alpha_list_usa[19:24],"SUBINDUSTRY", "USA", "TOP3000", "append")
generate_pending_simulation_data(fo_alpha_list[25:30],"SUBINDUSTRY", "ASI", "MINVOL1M", "append")
generate_pending_simulation_data(fo_alpha_list_usa[25:30],"SUBINDUSTRY", "USA", "TOP3000", "append")
generate_pending_simulation_data(fo_alpha_list[31:36],"SUBINDUSTRY", "ASI", "MINVOL1M", "append")
generate_pending_simulation_data(fo_alpha_list_usa[31:36],"SUBINDUSTRY", "USA", "TOP3000", "append")
generate_pending_simulation_data(fo_alpha_list[37:42],"SUBINDUSTRY", "ASI", "MINVOL1M", "append")
generate_pending_simulation_data(fo_alpha_list_usa[37:42],"SUBINDUSTRY", "USA", "TOP3000", "append")
"""


All simulation data written to output\alpha_list_pending_simulated_2_glb.csv in overwrite mode
All simulation data written to output\alpha_list_pending_simulated_2_glb_remain.csv in overwrite mode


'\ngenerate_pending_simulation_data(fo_alpha_list[30000:60000],"SUBINDUSTRY", "USA", "TOP3000", "OFF", "overwrite", "alpha_list_pending_simulated_8.csv")\ngenerate_pending_simulation_data(fo_alpha_list[60000:90000],"SUBINDUSTRY", "USA", "TOP3000", "OFF", "overwrite", "alpha_list_pending_simulated_10.csv")\ngenerate_pending_simulation_data(fo_alpha_list[90000:120000],"SUBINDUSTRY", "USA", "TOP3000", "OFF", "overwrite", "alpha_list_pending_simulated_12.csv")\ngenerate_pending_simulation_data(fo_alpha_list[120000:150000],"SUBINDUSTRY", "USA", "TOP3000", "OFF", "overwrite", "alpha_list_pending_simulated_14.csv")\ngenerate_pending_simulation_data(fo_alpha_list[150000:180000],"SUBINDUSTRY", "USA", "TOP3000", "OFF", "overwrite", "alpha_list_pending_simulated_15.csv")\ngenerate_pending_simulation_data(fo_alpha_list[180000:210000],"SUBINDUSTRY", "USA", "TOP3000", "OFF", "overwrite", "alpha_list_pending_simulated_16.csv")\ngenerate_pending_simulation_data(fo_alpha_list[210000:],"SUBINDUSTRY", "U

In [ ]:
generate_pending_simulation_data(fo_alpha_list,"SUBINDUSTRY", "USA", "TOP1000", "append")
generate_pending_simulation_data(fo_alpha_list,"SUBINDUSTRY", "USA", "TOP500", "append")
generate_pending_simulation_data(fo_alpha_list,"SUBINDUSTRY", "USA", "TOP200", "append")
generate_pending_simulation_data(fo_alpha_list,"SUBINDUSTRY", "USA", "TOPSP500", "append")

In [ ]:
# 文件路径
file_a_path = os.path.join('/Users/sujianan/py/alpha_simulator_bot/data', 'alpha_list_pending_simulated.csv')
file_b_path = os.path.join('/Users/sujianan/py/consultant/consultant/output', 'alpha_list_pending_simulated.csv')

# 合并文件
merge_csv_files(file_a_path, file_b_path, 'append')

In [ ]:
# Load alphas to task pools
fo_pools = load_task_pool(fo_alpha_list, 10, 10)
print(fo_pools[0])

## 7, 回测

In [ ]:
# Simulate First Order
multi_simulate(fo_pools, "SUBINDUSTRY", "USA", "TOP3000", 0)

## 8, 筛选Alpha


<div style="margin-left: 20px;">
1, get_alpha：截取有潜力提升表现至可以提交的alpha进入下一阶
</div>

<div style="margin-left: 20px;">
2, 剪枝Prune：精减相似alpha，提高回测资源利用率
</div>

In [ ]:
## get promising alphas to improve in the next order
fo_tracker = get_alphas("04-22", "04-23", 1.2, 0.7, "ASI", 5000, "track")
print(len(fo_tracker))
filtered_list = [
    item for item in fo_tracker
    if all(x not in item[1] for x in ['group', '\n', 'trade'])
]
print(len(filtered_list))
print(filtered_list[:3])

In [ ]:
fo_tracker = get_alphas_with_universe_region("04-09", "04-11", 1.2, 0.7, 'GLB', 300, "track")
filtered_list = [item for item in fo_tracker if '\n' not in item[1]]
print(len(filtered_list))
for item in filtered_list:
    print(item)

#### Prune 剪枝

In [ ]:
#fo_layer = prune(filtered_list, 'fnd17', 5)
fo_layer = digest(filtered_list)
# 剪枝后数量
print(len(fo_layer))
print(fo_layer[:3])

#fo_exp_layer = transform_expressions(fo_layer)
#print(len(fo_exp_layer))
#print(fo_exp_layer[:3])


In [ ]:
fo_layer = prune_with_universe(filtered_list, 'ts_backfill', 5, 'TOP3000')
# 剪枝后数量
print(len(fo_layer))
print(fo_layer[:3])

In [ ]:

# 主程序
deduped_alphas = deduplicate_alphas(filtered_list)
print(f"Original Alphas: {len(alphas)}, After Deduplication: {len(deduped_alphas)}")

param_combinations = generate_param_combinations()
print(len(param_combinations))
print(param_combinations[:3])
all_transformed_alphas = []

for alpha in deduped_alphas:
    for params in param_combinations:
        transformed_alpha = generate_transformed_alpha(alpha, **params)
        all_transformed_alphas.append(transformed_alpha)

unique_transformed_alphas = deduplicate_transformed_alphas(all_transformed_alphas)

# 输出前5个结果
for i, ta in enumerate(unique_transformed_alphas[:5]):
    print(f"Combination {i + 1}:")
    print(f"Alpha ID: {ta['alpha_id']}")
    print(f"Parameters: {ta['params']}")
    print(f"Transformed Expression:\n{ta['transformed_exp']}")
    print(f"Original Sharpe: {ta['original_sharpe']}, Original Fitness: {ta['original_fitness']}")
    print("-" * 70)

# 保存所有唯一结果
import json
with open('unique_transformed_alphas.json', 'w') as f:
    json.dump(unique_transformed_alphas, f, indent=4)
print(f"Total unique combinations generated: {len(unique_transformed_alphas)}")

## 9, 二阶提升
### ts_ops(field, days) -> group_ops(ts_ops(field, days), group)

In [ ]:
so_alpha_list = []
group_ops = ["group_neutralize", "group_rank", "group_zscore"]

for expr, decay in fo_layer:
#for expr, decay in fo_exp_layer:
    for alpha in get_group_second_order_factory([expr], group_ops, "ASI"):
        so_alpha_list.append((alpha,decay))

random.shuffle(so_alpha_list)
print(len(so_alpha_list))
print(so_alpha_list[:3])

In [ ]:
so_alpha_list = []
group_ops = ["group_neutralize", "group_rank", "group_zscore"]

for expr, decay in fo_layer:
#for expr, decay in fo_exp_layer:
    for alpha in get_group_second_order_factory_for_multi_line([expr], group_ops, "USA"):
        so_alpha_list.append((alpha,decay))

random.shuffle(so_alpha_list)
print(len(so_alpha_list))
print(so_alpha_list[:3])

In [ ]:
so_alpha_list = []
so_decay = 6
alpha_list = [item['transformed_exp'] for item in unique_transformed_alphas]
for alpha in alpha_list:
    so_alpha_list.append((alpha,so_decay))

random.shuffle(so_alpha_list)
print(len(so_alpha_list))
print(so_alpha_list[:3])

In [ ]:
df = get_datafields(s, dataset_id = 'pv29', region='USA', universe='TOP3000', delay=1)

group_field = df[df['type'] == 'GROUP']["id"].values.tolist()
print(group_field)
print(len(group_field))

so_alpha_list = []
group_ops = ["group_neutralize", "group_rank", "group_zscore"]

for expr, decay in fo_layer:
    lines = expr.strip().split('\n')
    # 获取最后一行并去除首尾空格
    last_line = lines[-1].strip()

    # 排除最后一行是 alpha 或 trade_alpha 的情况
    if last_line in ['alpha', 'trade_alpha']:
        continue
#for expr, decay in fo_exp_layer:
    for alpha in get_group_second_order_factory_for_multi_line_by_groups([expr], group_ops, group_field, "USA"):
        so_alpha_list.append((alpha,decay))

random.shuffle(so_alpha_list)
print(len(so_alpha_list))
print(so_alpha_list[:3])

In [ ]:
generate_pending_simulation_data(so_alpha_list,"SUBINDUSTRY", "ASI", "MINVOL1M", "append", 'alpha_list_pending_simulated_1.csv')

In [ ]:
# 文件路径
file_a_path = os.path.join('/Users/sujianan/py/alpha_simulator_bot/data', 'alpha_list_pending_simulated.csv')
file_b_path = os.path.join('/Users/sujianan/py/consultant/consultant/output', 'alpha_list_pending_simulated.csv')

# 合并文件
merge_csv_files(file_a_path, file_b_path, 'append')

### Simulate second order

In [ ]:
so_pools = load_task_pool(so_alpha_list, 10, 10)
multi_simulate(so_pools, 'SUBINDUSTRY', 'USA', 'TOP3000', 0)

## 10，三阶提升
group_ops(ts_ops(field, days), group) -> trade_when(entre_event, group_ops(ts_ops(field, days), group), exit_event)

In [ ]:
## get promising alphas from second order to improve in the third order
#so_tracker = get_alphas("04-22", "04-23", 1.3, 0.8, "ASI", 5000, "track")
so_tracker = get_alphas_from_csv("output/simulated_alphas_ASI.csv.2025-05-02", 1.3, 0.8)

filtered_list = [
    item for item in so_tracker
    if all(x not in item[1] for x in ['\n', 'trade'])
]
print(len(filtered_list))
print(filtered_list[:3])
so_layer = digest(filtered_list)
th_alpha_list = []


for expr, decay in so_layer:
    for alpha in trade_when_factory("trade_when",expr,"ASI"):
        th_alpha_list.append((alpha,decay))

random.shuffle(th_alpha_list)        
print("三阶表达式数量:%s"%len(th_alpha_list))

In [ ]:
## get promising alphas from second order to improve in the third order
so_tracker = get_alphas_with_universe_region("04-01", "04-04", 1.3, 0.8, "USA", 200, "submit")

so_layer = prune_with_universe(so_tracker, 'fnd6', 5, 'TOP500')
th_alpha_list = []

for expr, decay in so_layer:
    for alpha in trade_when_factory_for_multi_line("trade_when",expr,"USA"):
        th_alpha_list.append((alpha,decay))

random.shuffle(th_alpha_list)        
print("三阶表达式数量:%s"%len(th_alpha_list))
print(th_alpha_list[:3])

In [ ]:
generate_pending_simulation_data(th_alpha_list,"SUBINDUSTRY", "ASI", "MINVOL1M", "append", 'alpha_list_pending_simulated_1.csv')

### Simulate Third Order

In [ ]:
# Simulate third order
th_pools = load_task_pool(th_alpha_list, 10, 9)
multi_simulate(th_pools, 'SUBINDUSTRY', 'USA', 'TOP3000', 0)

## 11, 获取可提交的Alpha

<div style="margin-left: 20px;">
1, 拉取sharpe,fitness达到提交要求的alpha
</div>

<div style="margin-left: 20px;">
2, Check Submission：检查其他Test是否达到要求
</div>

<div style="margin-left: 20px;">
2, view_alphas 对可以提交的alpha进行排序
</div>


In [ ]:
# 1.58 sharpe, 1 fitness, "submit"参数
th_tracker = get_alphas("07-16", "07-20", 1.58, 1, "USA", 200, "submit")

In [ ]:
## 将get的alpha的id取出至stone_bag，用api check submission
stone_bag = []
for alpha in th_tracker:
    stone_bag.append(alpha[0])
print(len(stone_bag))
gold_bag = []
check_submission(stone_bag, gold_bag, 0)

In [ ]:
# 打印可提交的alpha信息并按sharpe排序，在网页上找到alpha手动提交
view_alphas(gold_bag)

## 12, 微调可以提交的alpha

<div style="margin-left: 20px;">
1, 得到更好的表现
</div>
<div style="margin-left: 40px;">
调整中性化，操作符参数，Decay
</div>

<div style="margin-left: 20px;">
2, Alpha质量评估
</div>

<div style="margin-left: 40px;">
performance comparison，turnover，margin
</div>

<div style="margin-left: 20px;">
3, 鲁棒性评估，防止过拟合
</div>

<div style="margin-left: 40px;">
更改中性化，Rank，Binary Test...
</div>

### Appendix

In [ ]:
# 模板构建Factory实例

def template_factory(sent_fields, option_fields):
    alpha_list = []
    for sent_field in sent_fields:
        for opt_field in option_fields:
            alpha_list.append("log(1+sigmoid(ts_zscore(%s,30))*sigmoid(ts_zscore(%s,30))"%(sent_field, opt_field))
    return alpha_list 

opt_df = get_datafields(s, dataset_id = 'option8', region='USA', universe='TOP3000', delay=1)
opt_fields = opt_df[opt_df['type'] == "MATRIX"]["id"].tolist()
print(opt_fields)

sent_df = get_datafields(s, dataset_id = 'sentiment1', region='USA', universe='TOP3000', delay=1)
sent_fields = sent_df[sent_df['type'] == "MATRIX"]["id"].tolist()
print(sent_fields)

alpha_list = template_factory(sent_fields, opt_fields)
print(alpha_list)